словарь с лексемами и их значениями из МАС

In [1]:
lexems = {
    "кисть": 
    [
        "кисть часть руки от запястья до конца пальцев", 
        "кисть скопление плодов или цветков на одной ветке, одном стебле; гроздь",
        "кисть пучок нитей, шнурков и т. п., служащий для украшения чего-л.",
        "кисть прикрепленный к рукоятке пучок щетины, волоса или синтетических волокон, служащий для нанесения краски, клея и т. п. на какую-л. поверхность"
    ], 
    "кора":
    [
        "кора наружная часть стеблей и корней древесных растений, легко отделяемая от древесины", 
        "кора твердый верхний слой, покров чего-л.",
        "кора наружная твердая оболочка земного шара, доступная исследованию",
        "кора поверхностный слой головного мозга у высших позвоночных животных и человека"
    ],
    "корень":
    [
        "корень подземная часть растения, посредством которой оно укрепляется в почве и получает из земли воду с растворенными в ней минеральными веществами", 
        "корень внутренняя, находящаяся в теле часть волоса, зуба, ногтя",
        "корень начало, источник, основа чего-л.",
        "корень основная часть слова (без приставок и суффиксов), которая не поддается дальнейшему разложению на значащие составные части",
        "корень число, дающее данное число при возведении его в определенную степень"
    ],
    "предложение":
    [
        "предложение действие по знач. глаг. предложить", 
        "предложение то, что предложено, предлагается",
        "предложение просьба мужчины, обращенная к женщине, вступить с ним в брак",
        "предложение поступление товаров на рынок",
        "предложение высказывание, являющееся сообщением о чем-л."
    ],
    "коса":
    [
        "коса заплетенные волосы", 
        "коса ручное сельскохозяйственное орудие для скашивания травы и злаков в виде длинного изогнутого ножа, отточенного с одной стороны и прикрепленного к длинной рукоятке",
        "коса длинная узкая отмель, идущая от берега, или низменный узкий мыс",
        "коса узкая полоса чего-л."
    ]
}

загрузка датасетов:

In [3]:
import wget

file = wget.download("https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/GIKRYA_texts_new.zip")
file3 = wget.download("https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/OpenCorpora_Texts.rar")

100% [..........................................................................] 2736818 / 2736818

In [4]:
import patoolib

patoolib.extract_archive(file)
patoolib.extract_archive(file3)

INFO patool: Extracting GIKRYA_texts_new.zip ...
INFO patool: running D:\7-Zip\7z.EXE x -o.\Unpack_ct53ybbv -- GIKRYA_texts_new.zip
INFO patool:     with creationflags=134217728, input=
INFO patool: ... GIKRYA_texts_new.zip extracted to `GIKRYA_texts_new' (multiple files in root).
INFO patool: Extracting OpenCorpora_Texts (1).rar ...
INFO patool: running D:\7-Zip\7z.EXE x -o.\Unpack_a7fbtu8u -- "OpenCorpora_Texts (1).rar"
INFO patool:     with creationflags=134217728, input=
INFO patool: ... OpenCorpora_Texts (1).rar extracted to `OpenCorpora_Texts (1)' (local file exists).


'OpenCorpora_Texts (1)'

In [5]:
from corus import load_morphoru_gicrya
from corus import load_morphoru_corpora

records_gikrya_train = load_morphoru_gicrya('gikrya_new_train.out')
records_gikrya_test = load_morphoru_gicrya('gikrya_new_test.out')
records_unamb = load_morphoru_corpora('unamb_sent_14_6.conllu')

так как я сразу загпузила лемматизированный корпус, остаётся только собрать контексты без пунктуации и стоп-слов 

In [9]:
import nltk
from string import punctuation
from simple_elmo import ElmoModel

from nltk.corpus import stopwords

stops = set(stopwords.words('russian'))

In [10]:
contexts = {
    "кисть": 
    [], 
    "кора":
    [],
    "корень":
    [],
    "предложение":
    [],
    "коса":
    []
}

In [11]:
set_lexemes = set(lexems.keys())

def contexts_extracting(records):
    for sent in records:
        lemmas = [w.lemma for w in sent.tokens if not w.lemma in stops and not w.lemma in punctuation]
        if set(lemmas) & set_lexemes: 
            intersect = list(set(lemmas) & set_lexemes)
            for lex in intersect: # на случай, если две лексемы встретились в одном предложении
                contexts[lex].append(lemmas)

In [12]:
rec_list = [records_gikrya_train, records_gikrya_test, records_unamb]

for g in rec_list:
    contexts_extracting(g)

загрузка модели

In [13]:
from simple_elmo import ElmoModel

model = ElmoModel()
model.load("196.zip")

2023-12-07 23:30:01,001 : INFO : Loading model from 196.zip...
2023-12-07 23:30:01,001 : INFO : 
            Assuming the model is a ZIP archive downloaded from the NLPL vector repository.
            Loading a model from a ZIP archive directly is slower than from the extracted files,
            but does not require additional disk space
            and allows to load from directories without write permissions.
            
2023-12-07 23:30:01,039 : INFO : We will cache the vocabulary of 100 tokens.
C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\simple_elmo\model.py:531: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  lstm_cell = tf.compat.v1.nn.rnn_cell.LSTMCell(


'The model is now loaded.'

векторизация

In [14]:
def get_elmo_vectors(word, contexts, model):
    all_vectors = model.get_elmo_vectors(contexts)
    word_vecs = []
    for i in range(len(contexts)):
        try:
            word_vecs.append(all_vectors[i][contexts[i].index(word)])
        except ValueError:
            continue
    return np.array(word_vecs)

In [15]:
vectors = {
    "кисть": 
    [], 
    "кора":
    [],
    "корень":
    [],
    "предложение":
    [],
    "коса":
    []
}

векторизация значений из словаря

In [16]:
vectors_definitions = {
    "кисть": 
    [], 
    "кора":
    [],
    "корень":
    [],
    "предложение":
    [],
    "коса":
    []
}

In [17]:
for key in lexems.keys():
    vectors_definitions[key] = get_elmo_vectors(key, contexts[key], model)

2023-12-07 23:33:46,669 : INFO : Warming up ELMo on 25 sentences...
2023-12-07 23:34:03,149 : INFO : Warming up finished.
2023-12-07 23:34:03,249 : INFO : Texts in the current batch: 25


NameError: name 'np' is not defined

для кластеризации я решила выбрать метод kmeans, наилучшее количество кластеров вычислить с помощью ElbowMethod

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
import numpy as np

model_KMeans = KMeans(random_state=42)

In [ ]:
for key in contexts.keys():
    vectors[key] = get_elmo_vectors(key, contexts[key], model)
    elb_visualizer = KElbowVisualizer(model_KMeans, k=(2, 15))
    elb_visualizer.fit(vectors[key])
    elb_visualizer.show()

In [ ]:
kmeans1 = KMeans(n_clusters=8).fit(vectors["кисть"])
kmeans2 = KMeans(n_clusters=7).fit(vectors["кора"])
kmeans3 = KMeans(n_clusters=11).fit(vectors["корень"])
kmeans4 = KMeans(n_clusters=6).fit(vectors["предложение"])
kmeans5 = KMeans(n_clusters=7).fit(vectors["коса"])

In [ ]:
kmeans1.labels_

In [182]:
import pandas as pd

df1 = pd.DataFrame({"контекст": contexts["кисть"], "значение": kmeans1.labels_})
df2 = pd.DataFrame({"контекст": contexts["кора"], "значение": kmeans2.labels_})
df3 = pd.DataFrame({"контекст": contexts["корень"], "значение": kmeans3.labels_})
df4 = pd.DataFrame({"контекст": contexts["предложение"], "значение": kmeans4.labels_})
df5 = pd.DataFrame({"контекст": contexts["коса"], "значение": kmeans5.labels_})

In [183]:
dfs = [df1, df2, df3, df4, df5]

In [187]:
for df in dfs:
    display(df.sort_values('значение'))

,контекст,значение
5,"[ствол, револьвер, кисть, рука, держащий, оруж...",0
2,"[Вот, делать, резкий, поворот, кончик, рапира,...",1
18,"[потереть, палец, сочленение, кисть]",1
17,"[касаться, масляный, краска, кисть]",1
8,"[холод, моментально, сковать, движение, художн...",1
0,"[показаться, синяк, лицо, начать, немного, схо...",2
22,"[механизм, лежать, блокировка, случай, путь, к...",2
21,"[чувствовать, странный, нежность, протянуть, р...",2
20,"[тщетно, искать, бледный, неподвижный, личико,...",2
15,"[дотянуться, нижний, кисть, коробочка, бомба, ...",2


,контекст,значение
3,"[кора, головной, мозг, прожигаться, новый, ней...",0
9,"[рано, упоминать, аксон, клетка, слой, 2, 3, в...",0
2,"[кора, дерево, проступать, размытый, искаженны...",1
6,"[съедобный, насекомое, живущий, кора, дерево, ...",1
7,"[ствол, дерево, толстый, несколько, обхват, пр...",1
15,"[каменный, кора, осыпаться, поблёскивать, белы...",1
4,"[Жанна, вцепиться, ствол, постараться, сделать...",2
8,"[—, поспособствовать, быстрый, заживление, рек...",3
11,"[следовать, предварительно, ошпарить, кора, пр...",3
1,"[взмахнуть, кусок, кора, едва, хлопнуть, нос, ...",4


,контекст,значение
24,"[профилактика, комплексный, терапия, онкологич...",0
36,"[ваш, малый, народ, стесняться, свой, настоящи...",1
43,"[в_общем, украинец, русский, корень, начать, д...",2
22,"[знать, свой, корень, это, здорово]",2
28,"[попытаться, кратко, напомнить, методологическ...",2
30,"[золотой, корень, успеть, многократно, прошить...",2
19,"[пропустить, главный, корень, должный, где-то,...",2
42,"[заявление, требоваться, большой, научный, лич...",2
3,"[называть, культура, —, половецкий, печенежски...",2
41,"[самый, дело, какой-то, температурный, приспос...",2


,контекст,значение
89,"[каждый, прочитанный, предложение, охватывать,...",0
83,"[так_что, понять, правильно, предложение, осно...",0
68,"[спасибо, мочь, нить, предложение, высказать]",0
40,"[ваш, предложение, просить, отправлять, почта,...",0
91,"[какой-то, предложение, дата, что-то, ещё, мил...",0
...,...,...
67,"[любимый, занятие, —, листать, книга, отзыв, п...",5
71,"[задумывать, кто-нибудь, мероприятие, походный...",5
73,"[крайне, важный, ваш, предложение, замечание]",5
75,"[руководитель, российский, регион, десятка, пр...",5


,контекст,значение
4,"[высокий, тонкий, Келл, длинный, колено, коса,...",0
10,"[сон, волос, необходимый, собрать, хвост, запл...",1
13,"[заплетать, длинный, коса, сложенный, корона, ...",1
3,"[стекать, необъятный, водопад, её, плечо, спин...",2
5,"[лицо, глаз, ожить, засверкать, подобно, свет,...",2
6,"[Кара, довольно, усмехнуться, закончить, плест...",2
18,"[впечатление, усугублять, коса, лопата, прочий...",3
1,"[маргаритка, небрежно, забросить, коса, спина,...",3
2,"[захлебываться, рыдание, снова, снова, взмахив...",3
17,"[проплыть, ещё, несколько, миля, разбить, лаге...",3
